本文件为生成汉化文件的源代码，如只想生成默认的汉化（即与Release内相同的汉化文件，只需要Pip安装Requirements.txt后依次运行下列代码，即可在build文件夹内生成所有汉化用文件
如想要自己修改汉化内容或添加字体，请根据文件内指示进行操作。

Huge thanks to stopnoanime for decrypting the dialog file of Highfleet , localization would never be possible without your work.

感谢 Homobanana,Iansniper,Commie-Spy,Rogo921,BI-XY,Xchiliarch,Eistin-Yite,OpaqueArc,KagaJiankui,Suesun-1132,xlmzg,LeberechtSchorner 参与汉化工作。

感谢所有参与提交汉化问题的群友。

This project can never be done by me alone.

In [1]:
import HF_Func as HF
import re


### 第一步 输入参数并填写fonts.txt
fonts.txt填写例子：A20{coorX,coorY,rectX,rectY} A20：字体名称，可使用除$*及游戏内使用的符号 每种字体输完换行

大括号内数据需生成字体Draw_char.txt后在绘图软件内实际情况填写但是必须含大括号及数字。未确定前可使用随机数填充 如 A20{0,0,0,0}


In [3]:
default_font = 'SH20_SAND'               #确定默认字体   #define the default font
row_num = 51                        #贴图一行字符数 #define the chars in a row in Photoshop
tex_name = 'Static11'               #材质贴图名称   #the name of the tex file
font_file ='fonts.txt'              #字体名字       #the fonts used in the dialog
translation_file = 'chinese.txt'    #汉化文件       #the translation of dialog

### 第二步 运行字体分析

In [28]:
HF.check_fonts(translation_file,font_file,default_font)     #汉化文件，字体参数，默认字体     #parameters       
f= open(font_file,encoding='utf-8')                         #check_fonts()生成各字体使用情况——除了默认字体      
font =  f.read().split('\n')
f.close()
fonts = []
for i in font:
    reg = re.match('(.*?){.*}',i)                     ## 如提示字符集发生变更，需要重新绘制字库贴图，否则游戏内无法应用更新！##
    if reg!= None:
        fonts.append(reg.groups()[0])  

for font_name in fonts:                                     #读取字体名称及参数
    print(f'\n{font_name}: ',end='')

    if font_name != default_font:                           #非默认字体，读取其usage.txt生成字符集
        globals()[f'{font_name}chars'] = HF.find_all_chara_text(f'.\\int_files\\usage\\{font_name}usage.txt',font_name)   #统计所有出现字符，返回字符list 生成 字符名称All_char.txt
    else:                                                   #默认字体直接读取汉化文件生成字符集，具有Fallback
        globals()[f'{font_name}chars'] = HF.find_all_chara_text(translation_file,font_name)

    HF.output_font(globals()[f'{font_name}chars'],row_num,font_name)                               #生成用于绘制贴图的txt  字体名称Draw_char.txt


SH20_SAND: 2176 chars 
SH20_BLUE: 11 chars 
SH20_YELLOW: 56 chars 
SH20_BLACK: 97 chars 
SH20_BROWN: 27 chars 
SH20_DIRT: 24 chars 
SH20_LSAND: 161 chars 
SH20_RED: 34 chars 
SH20_WHITE: 483 chars 
SH40_WHITE: 158 chars 
SH40_SAND: 76 chars 
SH40_SHADOW_WHITE: 91 chars 
SH40_LSAND: 3 chars 
SH65_SAND: 2 chars 
SH65_YELLOW: 8 chars 
MC20_BLACK: 227 chars 
PH20_SAND: 0 chars 
FP20_SAND: 0 chars 
SH16_WHITE: 50 chars 

### 第三步 根据build文件夹里生成的Draw_char.txt绘制贴图

在build文件夹内的psd文件内添加新的字体

此步需要在确定字体各参数：起始坐标X，起始坐标Y，字符宽度及字符高度并填入fonts.txt

教程参考https://steamcommunity.com/sharedfiles/filedetails/?id=2709021320

### 第四步 生成各字体Res

In [5]:
for font_name in fonts:                                     #读取字体名称及参数
    params = HF.font_params(font_file,font_name)
    coorX = int(params[1])                                  #起始坐标X
    coorY = int(params[2])                                  #起始坐标Y
    rectX = int(params[3])                                  #字符宽度
    rectY = int(params[4])                                  #字符高度

    name_lookup =[]
    try :
        HF.res_registration(globals()[f'{font_name}chars'],tex_name,font_name,coorX,coorY,rectX,rectY,row_num)   #生成 字体名称.res与字体lookup.txt
    except:
        print(f'{tex_name}_{font_name}.res error')
print(f'res generated')

res generated


### 第五步 生成总res、english.seria_enc

In [26]:
HF.res_compile(tex_name)        #生成Res
HF.encode_gen(translation_file,default_font) #生成编码后的english.txt

f= open('.\\int_files\\english.txt','rb')       #生成english.seria_enc 源代码来自https://github.com/stopnoanime/highfleet-dialog                
chars =  f.read()
f.close()
b = 2531011
f= open('.\\build\\english.seria_enc','wb')  
for item in chars:
    if item ==13:
        continue
    a = ((b ^ (b >> 15) ^ item) & 0xff)
    f.write(a.to_bytes(1,byteorder='little', signed=False))
    b += 214013
f.close()


### 清除变量

In [12]:
%reset -f

## test field